# Interactive vizualisation

In this Homework, we are going to use the spreadsheets from the Swiss National Science Foundation on funding per project per University or Research Institution. We will indentify the Cantons where the Institutions are and sum up the budget since the start of funding by the SNSF and produce a map whose color code corresponds to the total budget throughout the years.

At a second step, we will examine whether there is a difference in education and research investiment between the German and the French parts of Switzerland, divided by the so-called Röstigraben.

In [5]:
import folium
import json

import numpy as np
import pandas as pd
from helpers import *

We have directly downloaded the data in csv format from the SNSF website.

In [4]:
grants_data = pd.read_csv("P3_GrantExport.csv", sep=';')
grants_data.head(2)

,Project Number,Project Title,Project Title English,Responsible Applicant,Funding Instrument,Funding Instrument Hierarchy,Institution,University,Discipline Number,Discipline Name,Discipline Name Hierarchy,Start Date,End Date,Approved Amount,Keywords
0,1,Schlussband (Bd. VI) der Jacob Burckhardt-Biog...,NaN,Kaegi Werner,Project funding (Div. I-III),Project funding,NaN,Nicht zuteilbar - NA,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,11619.00,NaN
1,4,Batterie de tests à l'usage des enseignants po...,NaN,Massarenti Léonard,Project funding (Div. I-III),Project funding,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,10104,Educational science and Pedagogy,"Human and Social Sciences;Psychology, educatio...",01.10.1975,30.09.1976,41022.00,NaN


In what follows, we will work out the data frame in order to obtain a list of Institutions and the total budget.

In [9]:
# Selecting the relevant columns
data = grants_data[['University', 'Start Date','End Date', 'Approved Amount']]

# Selecting only years from the dates
data['Start Date'] = data['Start Date'].map(lambda x: str(x)[6:])
data['End Date'] = data['End Date'].map(lambda x: str(x)[6:])

data.head()

C:\Users\mariele\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\mariele\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,University,Start Date,End Date,Approved Amount
0,Nicht zuteilbar - NA,1975,1976,11619.00
1,Université de Genève - GE,1975,1976,41022.00
2,"NPO (Biblioth., Museen, Verwalt.) - NPO",1976,1985,79732.00
3,Universität Basel - BS,1975,1976,52627.00
4,"NPO (Biblioth., Museen, Verwalt.) - NPO",1976,1978,120042.00


In [10]:
#Selecting only 'university' and 'approved amount' which contains real data and excluding NPO (non-profit org)
data_amount = data[(data['University'].str.contains("N")==False) &(data['University'].str.contains("Nicht")==False) & (data['Approved Amount'].str.contains("not")==False)]

#Sort it by Start Date in ascendint order
data_am = data_amount.sort_values(ascending=[True], by=['Start Date'])
data_am.head()

,University,Start Date,End Date,Approved Amount
15501,Universität Zürich - ZH,,,46230.00
1905,Université de Genève - GE,1975,1976,95702.00
1904,Université de Genève - GE,1975,1976,1559894.00
1903,Université de Lausanne - LA,1975,1976,53465.00
1901,Université de Fribourg - FR,1975,1978,296625.00


In [11]:
# gonna drop the first row which has empty cell
data_am.drop(data_am.index[0],inplace=True)

#Convert cell content to numerical value and ranme 
data_am['Approved Amount'] = data_am['Approved Amount'].astype(float)

data_am.rename(columns={'Approved Amount': 'Amount'}, inplace=True)

In [65]:
# Group by university and sum over 'Aproved Amount' (it's the only numeric content, so it will only sum over this column)
data_uni = data_am.groupby('University')
total_sum = data_uni.sum()

total_sum.tail()

,Amount
University,
Université de Fribourg - FR,4.575262e+08
Université de Genève - GE,1.838237e+09
Université de Lausanne - LA,1.183291e+09
Weitere Spitäler - ASPIT,1.074981e+07
Zürcher Fachhochschule (ohne PH) - ZFH,4.484345e+07


### Matching the Universities to the Cantons
Now, going back to the full data. I wanna match the Universities to the Cantons. I'm gonna use a file found on the web with the abbreviations of Canton's names.

I'll be dealing already with the data frame with single entrance for universities and the total budget throughout the years.

In [66]:
cantons =  pd.read_csv('cantons.csv')
list_cantons =  cantons.code.values
list_cantons

array(['ZH', 'BE', 'LU', 'UR', 'SZ', 'OW', 'NW', 'GL', 'ZG', 'FR', 'SO',
       'BS', 'BL', 'SH', 'AR', 'AI', 'SG', 'GR', 'AG', 'TG', 'TI', 'VD',
       'VS', 'NE', 'GE', 'JU'], dtype=object)

In [67]:
uni_cantons = {k: get_canton(k, list_cantons) for k in total_sum.index if pd.notnull(k)}

### Getting coordinates of Universities

In [68]:
lat = pd.Series([0.0000*len(total_sum)], index = total_sum.index)
lon = pd.Series([0.0000*len(total_sum)], index = total_sum.index)

In [69]:
for university_name in total_sum.index:
        name_split = university_name.split('-')[0].strip() # omit abbreviations
        with open('api-key.txt', 'r') as api_file:
            api_key = api_file.read().replace('\n', '') # api key for Google API

        gmaps = googlemaps.Client(key=api_key)
        geocode_result = gmaps.geocode(name_split)
        if len(geocode_result) != 0:
            lat[university_name] = geocode_result[0]['geometry']['location']['lat']
            lon[university_name] = geocode_result[0]['geometry']['location']['lng']

In [70]:
total_sum2 = total_sum
total_sum2['Lat'] = lat
total_sum2['Lon'] = lon
total_sum2.head()

,Amount,Lat,Lon
University,,,
AO Research Institute - AORI,3435621.0,46.777120,9.813693
Allergie- und Asthmaforschung - SIAF,19169965.0,0.000000,0.000000
Berner Fachhochschule - BFH,31028695.0,47.057423,7.621388
Biotechnologie Institut Thurgau - BITG,2492535.0,47.648327,9.161303
Centre de rech. sur l'environnement alpin - CREALP,1567678.0,46.228559,7.367433


Checking which are the one for which we didn't find a match with googlemaps

In [71]:
#[k for k in uni_cantons.keys() if uni_cantons[k] is None]

Assigning the Cantons by hand

In [72]:
uni_cantons['AO Research Institute - AORI'] = 'GR'                       # Davos, GR
uni_cantons['Allergie- und Asthmaforschung - SIAF'] = 'GR'               # Davos, GR
uni_cantons['Berner Fachhochschule'] = 'BE'                              # Bern, BE
uni_cantons["Centre de rech. sur l'environnement alpin - CREALP"] = 'VS' # Sion, VS
uni_cantons['Eidg. Anstalt für Wasserversorgung - EAWAG'] = 'ZH'         # Dübendorf, ZH
uni_cantons['Eidg. Forschungsanstalt für Wald,Schnee,Land - WSL'] = 'ZH' # Birmensdorf, ZH
uni_cantons['Eidg. Hochschulinstitut für Berufsbildung - EHB'] = 'BE'    # Zollikofen, BE
uni_cantons['Eidg. Material und Prüfungsanstalt - EMPA'] = 'ZH'          # Dübendorf, ZH (also St Gallen & Thun...)
uni_cantons['Ente Ospedaliero Cantonale - EOC'] = 'TI'                   # Bellinzone, TI
uni_cantons['Fachhochschule Kalaidos - FHKD'] = 'ZH'                     # Zürich, ZH
uni_cantons['Fachhochschule Nordwestschweiz (ohne PH) - FHNW'] = 'SO'    # Olten, SO
uni_cantons['Fachhochschule Ostschweiz - FHO'] = 'SG'                    # St. Gallen, SG
uni_cantons['Fernfachhochschule Schweiz (Mitglied SUPSI) - FFHS'] = 'VS' # Brig, VS
uni_cantons['Forschungsinstitut für Opthalmologie - IRO'] = 'VS'         # Sion, VS
uni_cantons['Forschungsinstitut für biologischen Landbau - FIBL'] = 'AG' # Frick, AG
uni_cantons['Forschungskommission SAGW'] = 'BE'                          # Bern, BE
uni_cantons['Friedrich Miescher Institute - FMI'] = 'BS'                 # Basel, BS
uni_cantons['Haute école pédagogique du canton de Vaud - HEPL'] = 'VD'   # Lausanne, VD
uni_cantons['Haute école pédagogique fribourgeoise - HEPFR'] = 'FR'      # Fribourg, FR
uni_cantons['Idiap Research Institute - IDIAP'] = 'VS'                   # Martigny, VS
uni_cantons['Inst. Suisse de Spéléologie et Karstologie - ISSKA'] = 'NE' # La Chaux-de-Fonds, NE
uni_cantons['Inst. de Hautes Etudes Internat. et du Dév - IHEID'] = 'GE' # Geneva, GE
uni_cantons['Inst. universit. romand de Santé au Travail - IST'] = 'VD'  # Epalinges, VD
uni_cantons['Institut Universitaire Kurt Bösch - IUKB'] = 'VS'           # Bramois, VS
uni_cantons['Institut für Kulturforschung Graubünden - IKG'] = 'GR'      # Chur, GR
uni_cantons['Interkant. Hochschule für Heilpädagogik ZH - HfH'] = 'ZH'   # Zürich, ZH
uni_cantons['Physikal.-Meteorolog. Observatorium Davos - PMOD'] = 'GR'   # Davos, GR
uni_cantons['Pädagogische Hochschule Graubünden - PHGR'] = 'GR'          # Chur, GR
uni_cantons['Pädagogische Hochschule Nordwestschweiz - PHFHNW'] = 'BL'   # Liestal, BS
uni_cantons['Pädagogische Hochschule Wallis - PHVS'] = 'VS'              # Saint Maurice, VS
uni_cantons['Pädagogische Hochschule Zug - PHZG'] = 'ZG'                 # Zug, ZG
uni_cantons['Schweiz. Hochschule für Logopädie Rorschach - SHLR'] = 'SG' # Rorschach, SG
uni_cantons['Schweiz. Institut für Kunstwissenschaft - SIK-ISEA'] = 'ZH' # Zürich, ZH
uni_cantons['Schweizer Kompetenzzentrum Sozialwissensch. - FORS'] = 'VD' # Lausanne, VD

In [73]:
#[k for k in uni_cantons.keys() if uni_cantons[k] is None]

In [77]:
#making a list of canton abbreviations with index given by the university of the SNCF files

canton_column = pd.Series([0]*(len(total_sum)))
index_column = pd.Series([0]*(len(total_sum)))
i=0
for element in uni_cantons.keys():
    for element2 in total_sum.index:
        if element2 == element:
            canton_column[i] =uni_cantons[element]
            index_column[i]=element
            
            i = i+1
            
canton_column.index=index_column
canton_column.head()

Institut Universitaire Kurt Bösch - IUKB    VS
AO Research Institute - AORI                GR
Universität Luzern - LU                     LU
Université de Lausanne - LA                 VD
Facoltà di Teologia di Lugano - FTL         TI
dtype: object

## Merging Budjet and Canton information

In [78]:
# now let's merge the canton result of the search and the file with the budjet

uni_canton = pd.concat([canton_column, total_sum], axis=1)
uni_canton.rename(columns={0: 'Canton'}, inplace=True)
uni_canton.head()

,Canton,Amount,Lat,Lon
AO Research Institute - AORI,GR,3435621.0,46.777120,9.813693
Allergie- und Asthmaforschung - SIAF,GR,19169965.0,0.000000,0.000000
Berner Fachhochschule - BFH,BE,31028695.0,47.057423,7.621388
Biotechnologie Institut Thurgau - BITG,TG,2492535.0,47.648327,9.161303
Centre de rech. sur l'environnement alpin - CREALP,VS,1567678.0,46.228559,7.367433


In [79]:
#Excluding the row with Canton=None
uni_canton_final = uni_canton[uni_canton['Canton'].str.contains("None")==False]

uni_canton_final.head(6)

,Canton,Amount,Lat,Lon
AO Research Institute - AORI,GR,3.435621e+06,46.777120,9.813693
Allergie- und Asthmaforschung - SIAF,GR,1.916996e+07,0.000000,0.000000
Berner Fachhochschule - BFH,BE,3.102870e+07,47.057423,7.621388
Biotechnologie Institut Thurgau - BITG,TG,2.492535e+06,47.648327,9.161303
Centre de rech. sur l'environnement alpin - CREALP,VS,1.567678e+06,46.228559,7.367433
EPF Lausanne - EPFL,VD,1.175316e+09,46.519056,6.566758


I'll express the amount _per million euros_.

In [80]:
#mean = uni_canton_final['Amount'].mean()

normalize = lambda x: x/1000000

uni_canton_final['Amount(e+06)'] = uni_canton_final['Amount'].apply(normalize).round(5) # keeping up to 5 decimals



C:\Users\mariele\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Group by Canton and sum over the amount

In [81]:
canton_final = uni_canton_final[['Canton', 'Amount(e+06)']]  ## have to select only the amount, otherwise it will sum over lat and lon # this can be improved

canton_budjet = canton_final.groupby('Canton', as_index=False ) 
budjet = canton_budjet.sum().round(2)

budjet.head(2)
#len(budjet)

,Canton,Amount(e+06)
0,AG,122.71
1,BE,1555.15


In [17]:
#budjet.to_csv('canton_value.csv', encoding='utf-8', index= False)

### Matching the cantons in the in the cantons.csv file with the budjet data frame above

In [82]:
ct_list0 = pd.read_csv("cantons.csv")
ct_list = ct_list0.drop('number', axis=1) #drop the useless number column
ct_list.rename(columns={'code':'Canton'}, inplace=True) # rename the column to Canton
ct_list.index=ct_list.Canton # and make this the index column 'Canton'

budjet.index = budjet.Canton  # make the index to be the 'Canton' code in the budjet data frame
budjet_ = budjet.drop('Canton',axis=1) # and drop that column

# merge the two data frames. As I prepared the index column to be the same, it will merge contingent on the index as I want
budjet_ct = pd.concat([ct_list,budjet_],axis=1).fillna(0) # replace the NaN by zeros already

budjet_ct.to_csv('canton_value.csv', encoding='utf-8', index= False) # save it as csv to use to color the map in what follows

budjet_ct.head(5)  # take a look

,Canton,Amount(e+06)
AG,AG,122.71
AI,AI,0.00
AR,AR,0.00
BE,BE,1555.15
BL,BL,0.00


# Coloring the map

In [83]:
import json
import pandas as pd
import folium

with open('ch-cantons.topojson.json') as json_data:
    cover = json.load(json_data)

In [84]:
import numpy as np
import math
topo_path = r'ch-cantons.topojson.json'
grants_budjet = r'canton_value.csv'
budjet_c = pd.read_csv(grants_budjet)

color_scale = [1,600,1200,1800,2700,3600]

map = folium.Map(location=[46.8116, 8.3319], tiles='stamenterrain', zoom_start=8)

folium.TileLayer('cartodbpositron').add_to(map)


map.choropleth(geo_path=topo_path, data=budjet_c,
             columns=['Canton', 'Amount(e+06)'],               
             key_on='feature.id',
             fill_color='BuPu', fill_opacity=0.7, 
             line_opacity=0.2, topojson='objects.cantons',
             legend_name='Bujdet (%)',threshold_scale=color_scale)

map.save('map.html')
map

In [86]:
# adding the circles
map = folium.Map(location=[46.8116, 8.3319], tiles='stamenterrain', zoom_start=8)

folium.TileLayer('cartodbpositron').add_to(map)

map.choropleth(geo_path=topo_path, data=budjet_c,
             columns=['Canton', 'Amount(e+06)'],               
             key_on='feature.id',
             fill_color='BuPu', fill_opacity=0.7, 
             line_opacity=0.2, topojson='objects.cantons',
             legend_name='Bujdet (%)',threshold_scale=color_scale)

for i in range(0,len(total_sum2)):
    coord = [total_sum2['Lat'][i], total_sum2['Lon'][i]]
    amount = total_sum2['Amount'][i]/100000
    uni_name = total_sum2.index[i]
    legend = "{}, Budjet (/10^6 CHF) {}".format(uni_name,amount)
    folium.CircleMarker(location=coord, radius=amount, popup=legend,color='#3066cc',fill_color='#3066cc',fill_opacity=0.2).add_to(map)
map

# Is there a difference in funding between French and German Cantons?

Found a little table with the language of each canton online, so I'll scrape it and make a table

In [87]:
from bs4 import BeautifulSoup
import urllib.request as ur 
import json


base_url = 'http://schweizer-kantone-orte.websieb.info/'
#base_url = 'https://en.wikipedia.org/wiki/Cantons_of_Switzerland'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36" }

req = ur.Request(base_url, headers = headers)
page = ur.urlopen(req).read()
soup = BeautifulSoup(page,'lxml') 

In [88]:
import pandas as pd
df = pd.read_html(page, flavor='bs4')[4]

df = df.drop(0)
df = df.loc[:,[3,7]]
df.columns = ['Canton','Language']
df = df.drop(27)  # this is just a row for the whole country
df.head(7)

,Canton,Language
1,AG,d
2,AR,d
3,AI,d
4,BL,d
5,BS,d
6,BE,"d,f"
7,FR,"f,d"


In [89]:
dfm = df.merge(budjet_c, left_on='Canton', right_on='Canton', how='outer')
dfm.head(7)
#dfm.iloc[5,1]='d'
dfm.loc[dfm.Canton =='BE','Language']='d'
dfm.loc[dfm.Canton =='FR','Language']='f'
dfm.loc[dfm.Canton =='VS','Language']='f'
dfm.loc[dfm.Canton =='GR','Language']='d'
dfm.loc[dfm.Canton =='TI','Language']='d'


dfm

,Canton,Language,Amount(e+06)
0,AG,d,122.71
1,AR,d,0.00
2,AI,d,0.00
3,BL,d,0.00
4,BS,d,1392.48
5,BE,d,1555.15
6,FR,f,459.07
7,GE,f,1877.10
8,GL,d,0.00
9,GR,d,36.54


Now, we just sum the budget of the french and german speaking cantons

In [90]:
rosti = dfm.groupby('Language').sum()
rosti

,Amount(e+06)
Language,
d,7063.47
f,4768.09


And divide by the number of cantons to check the difference. There is a prejudice that the german part is better funded, more active economically, so let's see if this is reinforced by the data.

In [91]:
dfm.Language.value_counts()

d    20
f     6
Name: Language, dtype: int64

In [93]:
print(7063.47/10)
print(4768.09/6)

706.347
794.6816666666667


And we see the naive comparison of funding per number of institution works actually against the prejudice.